# Homework - Module 03

The goal of this homework is to create a simple training pipeline, use mlflow to track experiments and register best model, but use an orchestration tool for it.

We'll use [the same NYC taxi dataset](https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page), the **Yellow** taxi data for March, 2023. 

### Question 1. Select the Tool

We will use the same tool you used when completing the module: `Prefect`.

### Question 2. Version

The version of our orchestrator is:

In [1]:
# Prefect version
!prefect --version

3.4.6


### Question 3. Creating a pipeline

Let's download and read the March 2023 Yellow taxi trips data.

In [2]:
# Download March data
!curl -O https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-03.parquet

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 53.5M  100 53.5M    0     0  1197k      0  0:00:45  0:00:45 --:--:-- 1183k


In [3]:
# List data file
!ls -lh yellow*

-rw-r--r--@ 1 cm-mboulou-mac  staff    54M Jun 13 06:37 yellow_tripdata_2023-03.parquet


In [4]:
# Necessary import
import pandas as pd
import numpy as np
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import root_mean_squared_error

# Python version
!python --version

Python 3.9.18


In [5]:
# Read the data
df = pd.read_parquet("yellow_tripdata_2023-03.parquet")
# Number of observations
print(f"Number of rows: {len(df)}.")

Number of rows: 3403766.



### Question 4. Data preparation

Let's continue with pipeline creation.

We will use the same logic for preparing the data we used previously. 

This is what we used (adjusted for yellow dataset):

```python
def read_dataframe(filename):
    df = pd.read_parquet(filename)

    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df.duration = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df
```

Let's apply to the data we loaded in question 3. 

What's the size of the result? 

- 2,903,766
- 3,103,766
- 3,316,216 
- 3,503,766

In [6]:
# Function for reading the data
def read_dataframe(filename):
    # Read the parquet data file
    df = pd.read_parquet(filename)

    # Feature engineering for creating the duration column
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    # Convert durations into minutes
    df.duration = df.duration.dt.total_seconds() / 60

    # Filtering for trips lasting from 1 min to 1 hour
    df = df[(df.duration >= 1) & (df.duration <= 60)]

    # Categorical features
    categorical = ['PULocationID', 'DOLocationID']
    # Convert categorival variables to string
    df[categorical] = df[categorical].astype(str)

    # return the data
    return df

In [7]:
# Read the data
df = read_dataframe("yellow_tripdata_2023-03.parquet")
# Number of observations
print(f"Number of rows: {len(df)}.")

Number of rows: 3316216.


### Question 5. Train a model

We will now train a linear regression model using the same code as in homework 1.

* Fit a dict vectorizer.
* Train a linear regression with default parameters.
* Use pick up and drop off locations separately, don't create a combination feature.

Let's now use it in the pipeline. We will need to create another transformation block, and return both the dict vectorizer and the model.

What's the intercept of the model? 

Hint: print the `intercept_` field in the code block

- 21.77
- 24.77
- 27.77
- 31.77

### Question 6. Register the model 

The model is trained, so let's save it with MLFlow.

Find the logged model, and find MLModel file. What's the size of the model? (`model_size_bytes` field):

* 14,534
* 9,534
* 4,534
* 1,534

---